### Generating cell type signatures for the deconvolution of DPD data

The signature generation is based on the code by Cobos et al. (benchmark on deconvolution) with some modifications:\
Link to paper: https://www.nature.com/articles/s41467-020-19015-1 \
Link to Github: https://github.com/favilaco/deconv_benchmark

**Notes on Running the Notebook:**
1) Make sure that you have R installed before running this notebook.
2) The required libraries that need to be installed are listed in the cell below.

In [5]:
library(data.table)
library(dplyr)
library(Matrix)
library(limma)
library(edgeR)
library(matrixStats)
require(plyr)

#install.packages("corrplot")
library(corrplot)
source("http://www.sthda.com/upload/rquery_cormat.r")
source('./helper_functions.R')  # https://github.com/favilaco/deconv_benchmark

In [6]:
# function for rpkm normalization
rpkm_normalize <- function(data){
    load("./dpd_data/exonicLength.rda") 

    data <- data[which(rownames(data) %in% rownames(exonicLength)),]  # note 1: counts must have EnsID as rownames

    # Format the exonicLength matrix/list
    length <- transpose(exonicLength)[[1]] # note 2: the file "exonicLength" must be loaded into the global environment,
    names(length) <- rownames(exonicLength)

    # Stats
    m <- match(rownames(data), names(length))
    length <- length[m]/1000 # per kilo base 
    # for each single cell sample the counts are summed up
    libsize <- apply(data, 2, sum) / 10^6 # per million mapped reads 

    # Normalise for library size, then length
    tpm <- data 
    for (j in c(1:ncol(tpm))) tpm[,j] <- data[,j]/libsize[j]

    rpkm <- tpm 
    for (j in c(1:ncol(rpkm))) rpkm[,j] <- tpm[,j]/length

    data <- rpkm

    data <- data.matrix(data)
    return(data)
}

In [7]:
########################################################
###      loading the processed single cell data      ###
### (non-affected (healthy) and affected cells)      ###
########################################################

# If adata of the paper is used in generating `all_proc` file then 
# load data from `./dpd_data/sc_preprocess/processed_data/all_proc`.
# If adata is generated from scratch load data from `./dpd_results/sc_preprocess/processed_data/all_proc`

# this may take some time
data= read.table('./dpd_data/sc_preprocess/processed_data/all_proc', sep= "\t", header= FALSE)
colnames(data) <- data[1,]
data <- data[-c(1), ]
data <- t(as.matrix(data)) # data: genes * samples
colnames(data) <- data[1,]
data <- data[-c(1), ] # because in loading the data, the headers are elements of the dataframe

data_ <- data.frame(apply(data, 2, function(x) as.integer(as.character(x))))
rownames(data_) <- rownames(data)
data <- data_
colnames(data) <- gsub("\\.", "-", colnames(data)) # "-" is converted to "." when reading the data in R                        
                          
dim(data)
head(data)

[1] 13487 29707

,GCCATGGTCGTTCTCG-4,GCTGGGTCACGGCCAT-1,ATTCACTTCCCTTGGT-5,CGTAATGAGTGGTCAG-5,CCACACTTCTCCAAGA-5,TGCCGAGTCTATACGG-2,AGACCATCAGCAGACA-5,GGGACAAAGCGCTGCT-5,GACACGCAGTTATGGA-4,CTCTCGAGTTCGAAGG-6,⋯,GACATCACAGAAGTTA-5,CTAGACATCTAAGGAA-5,ACTACGACAACAGCCC-4,TAGAGTCCAGGCTTGC-5,TGGTGATAGCTAGATA-6,GTCCCATCATTGAAAG-6,AAGAACATCTTAATCC-4,TCCCACACAATAGAGT-5,TTCCTTCAGTATTCCG-4,CGCATAAAGCCTCTTC-5
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
AL627309.1,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,1,0,0,0,0,1,0
AP006222.2,0,0,2,0,0,0,0,0,0,2,⋯,0,0,0,0,0,0,0,0,1,0
RP11-206L10.3,0,0,0,0,0,0,0,0,0,0,⋯,0,0,1,0,0,0,0,0,0,0
RP11-206L10.2,0,1,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
RP11-206L10.9,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
LINC00115,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [9]:
#############################################################
### Map cellType/cluster indices  to annotated cell types ###
#############################################################

# If adata of the paper is used in generating `dpd_phenoData` file then 
# load data from `./dpd_data/sc_preprocess/processed_data/dpd_phenoData`.
# If adata is generated from scratch load data from `./dpd_results/sc_preprocess/processed_data/dpd_phenoData`

# Cell type annotation is done before running this cell
full_phenoData = read.table('./dpd_data/sc_preprocess/processed_data/dpd_phenoData', header=TRUE)
print(unique(full_phenoData$cellType))
full_phenoData$cellType <- mapvalues(full_phenoData$cellType, 
                                     from=c(0, 1, 2, 3, 4, 8, 9, 10, 12, 13, 16, 15, 7, 14), 
                                     to=c("CN_DPD","CN_DPD","CN_DPD","AS_DPD","CN_H","CN_H",
                                          "NEU","AS_H","PGC","CBC","CBC", "NEC", "INTER", "BRC"))
print(unique(full_phenoData$cellType))
head(full_phenoData)
unique(full_phenoData$cellType)

 [1]  6  8  0 11 10  9  3  1  5 14  2  4 12 15  7 13 16
 [1] "6"      "CN_H"   "CN_DPD" "11"     "AS_H"   "NEU"    "AS_DPD" "5"     
 [9] "BRC"    "PGC"    "NEC"    "INTER"  "CBC"   


,cellID,cellType,sampleID
,<chr>,<chr>,<chr>
1,GCCATGGTCGTTCTCG-4,6,dpd
2,GCTGGGTCACGGCCAT-1,CN_H,h
3,ATTCACTTCCCTTGGT-5,CN_DPD,dpd
4,CGTAATGAGTGGTCAG-5,CN_DPD,dpd
5,CCACACTTCTCCAAGA-5,11,dpd
6,TGCCGAGTCTATACGG-2,AS_H,h


[1] "6"      "CN_H"   "CN_DPD" "11"     "AS_H"   "NEU"    "AS_DPD" "5"     
 [9] "BRC"    "PGC"    "NEC"    "INTER"  "CBC"

In [10]:
dataset= 'dpd_results/signatures'
to_remove='none'
deconv_type= 'bulk'
transformation= 'none'
normalization= 'none'
marker_strategy= 'all'

dir.create(dataset, recursive= TRUE, showWarnings = FALSE)

In [11]:
#####################
### Additional QC ###
#####################

require(dplyr); require(Matrix)

# First: cells with library size, mitochondrial or ribosomal content further than three MAD away were discarded
filterCells <- function(filterParam){
    cellsToRemove <- which(filterParam > median(filterParam) + 3 * mad(filterParam) | filterParam < median(filterParam) - 3 * mad(filterParam) )
    cellsToRemove
}

libSizes <- colSums(data)
gene_names <- rownames(data)

mtID <- grepl("^MT-|_MT-", gene_names, ignore.case = TRUE)
rbID <- grepl("^RPL|^RPS|_RPL|_RPS", gene_names, ignore.case = TRUE)

mtPercent <- colSums(data[mtID, ])/libSizes
rbPercent <- colSums(data[rbID, ])/libSizes

lapply(list(libSizes = libSizes, mtPercent = mtPercent, rbPercent = rbPercent), filterCells) %>% 
        unlist() %>% 
        unique() -> cellsToRemove

if(length(cellsToRemove) != 0){
    data <- data[,-cellsToRemove]
    full_phenoData <- full_phenoData[-cellsToRemove,]
}

# Keep only "detectable" genes: at least 5% of cells (regardless of the group) have a read/UMI count different from 0
keep <- which(Matrix::rowSums(data > 0) >= round(0.05 * ncol(data)))
data = data[keep,]

dim(data)
head(data)

[1] 10061 27465

,GCCATGGTCGTTCTCG-4,GCTGGGTCACGGCCAT-1,ATTCACTTCCCTTGGT-5,CGTAATGAGTGGTCAG-5,CCACACTTCTCCAAGA-5,TGCCGAGTCTATACGG-2,AGACCATCAGCAGACA-5,GGGACAAAGCGCTGCT-5,GACACGCAGTTATGGA-4,CTCTCGAGTTCGAAGG-6,⋯,AATCACGAGCAAACAT-5,CCTCAGTGTGGTACAG-5,CCTTCAGCACGAAAGC-4,GACATCACAGAAGTTA-5,CTAGACATCTAAGGAA-5,TAGAGTCCAGGCTTGC-5,GTCCCATCATTGAAAG-6,AAGAACATCTTAATCC-4,TCCCACACAATAGAGT-5,CGCATAAAGCCTCTTC-5
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
AL627309.1,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,1,0,0,0,0
AP006222.2,0,0,2,0,0,0,0,0,0,2,⋯,0,0,0,0,0,0,0,0,0,0
NOC2L,0,0,2,0,0,0,0,1,0,0,⋯,0,0,0,1,0,0,0,1,0,1
HES4,0,1,1,2,3,0,0,2,1,4,⋯,1,2,2,0,0,4,0,0,1,2
ISG15,0,0,0,0,0,1,0,0,0,0,⋯,0,1,1,0,0,0,0,0,0,0
AGRN,0,1,0,0,0,0,1,1,0,0,⋯,0,0,0,0,1,0,0,0,0,0


In [12]:
#################
### Add ensID ###
#################

# first convert mixture data to dataframe to do rpkm normalization
data <- as.data.frame(data) # conversion from matrix to dataframe

load(file = "dpd_data/geneInfo.rda")
geneInfo <- filt_gene_info

annot <- geneInfo[which(geneInfo$Biotype == "protein_coding"),]
# put ENSG at the end of ENSTs
# the following works as grouping by gene symbol and then sorting by ensID
annot <- annot[order(annot$Gene.Symbol, annot$ensID, decreasing = TRUE),]
dim(data)

sharedAnnotation <- annot[which(annot$Gene.Symbol %in% rownames(data)),]
matches <- match(sharedAnnotation$Gene.Symbol, rownames(data)) # matches has length 155483

# Add ensID to a new column
data$ensID <- "-"
data$ensID[matches] <- sharedAnnotation$ensID
data <- data[which(data$ensID != "-"),]

# Put ensID into rownames
rownames(data) <- data$ensID
data <- data[,-which(colnames(data) %in% "ensID")]

# removing the sub index (after .), example: ENSG00000121410.7 => ENSG00000121410
rownames(data) <- sapply(strsplit(rownames(data),"\\."), `[`, 1)
data <- data.matrix(data) # this was supposed to be after rpkm-normalization but normalization is skipped 

[1] 10061 27465

In [13]:
# added for dpd data:
# filtering the data to be limited to the cells in the intersection of full phenoData and data (not needed here)
cell_subset <- intersect(full_phenoData$cellID, colnames(data))
data <- data[, cell_subset]
full_phenoData <- full_phenoData[full_phenoData$cellID %in% cell_subset, ]
head(data)

,GCCATGGTCGTTCTCG-4,GCTGGGTCACGGCCAT-1,ATTCACTTCCCTTGGT-5,CGTAATGAGTGGTCAG-5,CCACACTTCTCCAAGA-5,TGCCGAGTCTATACGG-2,AGACCATCAGCAGACA-5,GGGACAAAGCGCTGCT-5,GACACGCAGTTATGGA-4,CTCTCGAGTTCGAAGG-6,⋯,AATCACGAGCAAACAT-5,CCTCAGTGTGGTACAG-5,CCTTCAGCACGAAAGC-4,GACATCACAGAAGTTA-5,CTAGACATCTAAGGAA-5,TAGAGTCCAGGCTTGC-5,GTCCCATCATTGAAAG-6,AAGAACATCTTAATCC-4,TCCCACACAATAGAGT-5,CGCATAAAGCCTCTTC-5
ENSG00000237683,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,1,0,0,0,0
ENSG00000188976,0,0,2,0,0,0,0,1,0,0,⋯,0,0,0,1,0,0,0,1,0,1
ENSG00000188290,0,1,1,2,3,0,0,2,1,4,⋯,1,2,2,0,0,4,0,0,1,2
ENSG00000187608,0,0,0,0,0,1,0,0,0,0,⋯,0,1,1,0,0,0,0,0,0,0
ENSG00000188157,0,1,0,0,0,0,1,1,0,0,⋯,0,0,0,0,1,0,0,0,0,0
ENSG00000078808,0,0,1,1,1,2,1,0,0,0,⋯,0,0,0,0,0,3,0,0,0,0


In [16]:
set.seed(24)
require(limma); require(dplyr); require(pheatmap)

# get the count of cells per cell type
original_cell_names = colnames(data)
colnames(data) <- as.character(full_phenoData$cellType[match(colnames(data),full_phenoData$cellID)])
table(colnames(data))

Loading required package: pheatmap




    11      5      6 AS_DPD   AS_H    BRC    CBC CN_DPD   CN_H  INTER    NEC 
  1141   1939   1954   2391   1323    660    537   9290   3581   1433    495 
   NEU    PGC 
  1502   1219 

In [17]:
# Keep cell type with >= 49 cells after QC
cell_counts = table(colnames(data))
to_keep = names(cell_counts)[cell_counts >= 49]
to_keep

pData <- full_phenoData[full_phenoData$cellType %in% to_keep,]
to_keep = which(colnames(data) %in% to_keep)   
data <- data[,to_keep]
original_cell_names <- original_cell_names[to_keep]

[1] "11"     "5"      "6"      "AS_DPD" "AS_H"   "BRC"    "CBC"    "CN_DPD"
 [9] "CN_H"   "INTER"  "NEC"    "NEU"    "PGC"

In [18]:
############################
### Limit the Cell Types ###
############################

my_CT_sub= c('CN_H','AS_H', 'CBC', 'INTER', 'PGC', 'BRC', 'NEU', '11', 'NEC')
pData <- pData[pData$cellType %in% my_CT_sub,]
my_CT_sub= which(colnames(data) %in% my_CT_sub)
data <- data[,my_CT_sub]
original_cell_names <- original_cell_names[my_CT_sub]

In [19]:
# Data split into train & test (they are the same in here)
training <- as.numeric(unlist(sapply(unique(colnames(data)), function(x) {
            sample(which(colnames(data) %in% x), cell_counts[x]) })))
#testing <- as.numeric(unlist(sapply(unique(colnames(data)), function(x) {
#            sample(which(colnames(data) %in% x), cell_counts[x]) })))

# Generate phenodata for reference matrix C
pDataC = pData[training,]

train <- data[,training]
#test <- data[,testing] # test is not used anywhere

In [20]:
head(train)

,CN_H,CN_H,CN_H,CN_H,CN_H,CN_H,CN_H,CN_H,CN_H,CN_H,⋯,CBC,CBC,CBC,CBC,CBC,CBC,CBC,CBC,CBC,CBC
ENSG00000237683,0,0,0,0,0,0,0,0,0,0,⋯,0,1,1,0,0,0,0,0,0,0
ENSG00000188976,1,0,0,0,1,0,0,0,0,0,⋯,0,1,2,0,0,0,0,0,0,0
ENSG00000188290,1,1,3,0,2,0,3,0,4,0,⋯,1,0,1,0,1,0,0,0,0,0
ENSG00000187608,0,0,0,0,0,0,0,0,0,0,⋯,1,0,1,1,0,0,0,0,0,0
ENSG00000188157,0,0,0,0,0,1,1,1,0,0,⋯,0,1,0,0,0,0,1,0,0,0
ENSG00000078808,1,0,0,0,0,0,0,0,0,0,⋯,2,0,0,1,1,3,0,0,0,1


In [21]:
# "write.table" & "saveRDS" statements are optional, for users willing to avoid generation of matrix C every time:    
write.table(pDataC, file = paste0(dataset,"/phenoDataC"),row.names=FALSE,col.names=TRUE,sep="\t",quote=FALSE)

train_cellID = train
colnames(train_cellID) = original_cell_names[training]

# reference matrix (C) + refProfiles.var from TRAINING dataset
cellType <- colnames(train)
group = list()
for(i in unique(cellType)){ 
    group[[i]] <- which(cellType %in% i)
}
C = lapply(group,function(x) Matrix::rowSums(train[,x])) # C should be made with the mean (not sum) to agree with the way markers were selected
C = do.call(cbind.data.frame, C)

refProfiles.var = lapply(group,function(x) train[,x])
refProfiles.var = lapply(refProfiles.var, function(x) matrixStats::rowSds(Matrix::as.matrix(x)))
refProfiles.var = round(do.call(cbind.data.frame, refProfiles.var))
rownames(refProfiles.var) <- rownames(train)

In [23]:
#####################################
### rpkm-normalize the signatures ###
#####################################

C_norm <- as.data.frame(rpkm_normalize(C))
C_norm$EnsID <- rownames(C_norm)
col_order <- c('EnsID', c('CN_H','AS_H', 'CBC', 'INTER', 'PGC', 'BRC', 'NEU', '11', 'NEC'))
C_norm <- C_norm[, col_order]
C_norm <- C_norm[order(C_norm$EnsID),]
head(C_norm)
write.table(C_norm, file = paste0(dataset, "/dpd_sig"),row.names=FALSE,col.names=TRUE,sep="\t",quote=FALSE)

,EnsID,CN_H,AS_H,CBC,INTER,PGC,BRC,NEU,11,NEC
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000000003,ENSG00000000003,4.418646,23.311192,40.349842,18.519446,9.8023919,8.9720072,15.932370,4.2086191,16.7941959
ENSG00000000419,ENSG00000000419,30.137975,37.396212,18.626105,20.118966,26.3358257,17.0613335,30.364764,27.0308373,39.7508793
ENSG00000001036,ENSG00000001036,5.041867,12.827187,15.594200,12.803148,7.7731910,4.6672464,7.259739,1.3150311,11.2808042
ENSG00000001084,ENSG00000001084,3.015123,3.407500,4.572194,3.388605,2.8806894,1.7200924,1.729770,0.5287077,1.2598451
ENSG00000001167,ENSG00000001167,3.160952,2.392714,1.928572,1.861021,2.2619512,0.4658255,3.454371,2.1035761,3.7769072
ENSG00000001460,ENSG00000001460,1.993394,1.071016,4.240126,1.222369,0.7119034,1.0008520,0.482424,0.2408728,0.5635343
